In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from pandas import ExcelWriter
# from pandas import ExcelFile
import sqlalchemy
# import sqlite3 as sq
import os

In [2]:
# print (os.environ)
if not os.environ.get('DYNO'):
    import config
    print(config.name)
    
if os.environ.get("DATABASE_URL"):
    url=os.environ["DATABASE_URL"]
else:
    url = config.url

engine= sqlalchemy.create_engine(url)

GlobalEnergy


In [3]:
total_consumption_df = pd.read_sql("SELECT * FROM total_consumption", engine)
emissions_df = pd.read_sql("SELECT * FROM emissions", engine)
coal_consumption_df = pd.read_sql("SELECT * FROM coal_consumption", engine)
electricity_consumption_df = pd.read_sql("SELECT * FROM electricity_consumption", engine)
ng_consumption_df = pd.read_sql("SELECT * FROM ng_consumption", engine)
oil_consumption_df = pd.read_sql("SELECT * FROM oil_consumption", engine)
electricity_renewables_df = pd.read_sql("SELECT * FROM electricity_renewables_share", engine)
gdp_df = pd.read_sql("SELECT * FROM gdp_table", engine)

In [4]:
df = total_consumption_df.copy()
df = df.loc[df["Year"]==max(df["Year"])]
df

,Year,World,OECD,G7,BRICS,Europe,European Union,Belgium,Czech Rep.,France,...,Africa,Algeria,Egypt,Nigeria,South Africa,Middle-East,Iran,Kuwait,Saudi Arabia,United Arab Emirates
27,2017,14126,5363,3804,5214,1857,1611,56,44,243,...,805,53,86,140,140,786,253,35,224,74


In [5]:
def funRankCountry(df_data,enertype,list_Col_NOT,country):
    df = df_data.copy()
    df = df.loc[df["Year"]==max(df["Year"])]
    list_Col_2Ext = [x for x in df.columns if x not in list_Col_NOT]
    
    list_dict = []
    for x in list_Col_2Ext:
        dict_x = {'Country': x, 'Type' : enertype,'Value': df[x].values[0]}
        list_dict.append(dict_x)
        
    list_dict_sorted = sorted(list_dict, key=lambda k: k['Value'], reverse=True)
    for i in range(len(list_dict_sorted)):
        i += 1
        list_dict_sorted[i-1]['Rank'] = i
        
    list_dict_x = [dict_x for dict_x in list_dict_sorted if dict_x['Country'] == country]
        
    return list_dict_x[0]

In [6]:
def funPreprocGDP(df_GDP,df_Name,list_Col_NOT):
    list_countries = list(df_GDP["Country"].unique())
    count = 0
    for country in list_countries:
        df_country = df_GDP.loc[df_GDP["Country"]==country][["Year","Total"]]
        df_country.rename(columns={'Total': country}, inplace=True)
        if count == 0:
            df_final = df_country
            count += 1
        else:
            df_final = pd.merge(df_final, df_country, on='Year', how='inner')
            
    df = df_Name.copy()    
    list_Col_2Ext = [x for x in df.columns if x not in list_Col_NOT]    
    list_Col_GDP = list(df_final.columns)
    
    new_list_GDP = []
    old_list_GDP = []
    for x in list_Col_2Ext:
        for y in list_Col_GDP:
            if (x in y) & (len(y)>len(x)):
                new_list_GDP.append(x)
                old_list_GDP.append(y)

    for i in range(len(new_list_GDP)):
        df_final = df_final.rename(columns={old_list_GDP[i]: new_list_GDP[i]})
    
            
    return df_final

In [7]:
country = "China"
list_Col_NOT = ["Year","World","OECD","G7","BRICS","Europe","European Union","Africa","Middle-East","CIS", \
                "Latin America","America","North America","Asia","Pacific"]
print(funRankCountry(total_consumption_df,"Total",list_Col_NOT,country))
print(funRankCountry(coal_consumption_df,"Coal",list_Col_NOT,country))
print(funRankCountry(electricity_consumption_df,"Electricity",list_Col_NOT,country))
print(funRankCountry(ng_consumption_df,"Natural Gas",list_Col_NOT,country))
print(funRankCountry(oil_consumption_df,"Oil",list_Col_NOT,country))
print(funRankCountry(electricity_renewables_df,"Electr Renew",list_Col_NOT,country))
df_GDP = funPreprocGDP(gdp_df,total_consumption_df,list_Col_NOT)
print(funRankCountry(df_GDP,"GDP",list_Col_NOT,country))

{'Country': 'China', 'Type': 'Total', 'Value': 3105, 'Rank': 1}
{'Country': 'China', 'Type': 'Coal', 'Value': 3607, 'Rank': 1}
{'Country': 'China', 'Type': 'Electricity', 'Value': 5683, 'Rank': 1}
{'Country': 'China', 'Type': 'Natural Gas', 'Value': 238, 'Rank': 3}
{'Country': 'China', 'Type': 'Oil', 'Value': 554, 'Rank': 2}
{'Country': 'China', 'Type': 'Electr Renew', 'Value': 25.97, 'Rank': 17}
{'Country': 'China', 'Type': 'GDP', 'Value': 10200000000000.0, 'Rank': 2}


In [8]:
country = "United States"
list_Col_NOT = ["Year","World","OECD","G7","BRICS","Europe","European Union","Africa","Middle-East","CIS", \
                "Latin America","America","North America","Asia","Pacific"]
df_GDP = funPreprocGDP(gdp_df,total_consumption_df,list_Col_NOT)
dict_final ={
    "Total" : funRankCountry(total_consumption_df,"Total",list_Col_NOT,country)["Rank"],
    "Coal" : funRankCountry(coal_consumption_df,"Coal",list_Col_NOT,country)["Rank"],
    "Electricity" : funRankCountry(electricity_consumption_df,"Electricity",list_Col_NOT,country)["Rank"],
    "Natural Gas" : funRankCountry(ng_consumption_df,"Natural Gas",list_Col_NOT,country)["Rank"],
    "Oil" : funRankCountry(oil_consumption_df,"Oil",list_Col_NOT,country)["Rank"],
    "Electr Renew": funRankCountry(electricity_renewables_df,"Electr Renew",list_Col_NOT,country)["Rank"],
    "GDP" : funRankCountry(df_GDP,"GDP",list_Col_NOT,country)["Rank"]
}
dict_final

{'Total': 2,
 'Coal': 3,
 'Electricity': 2,
 'Natural Gas': 1,
 'Oil': 1,
 'Electr Renew': 22,
 'GDP': 1}